In [1]:
import pickle, random, copy, platform, os
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [2]:
if platform.system() == 'Windows': 
    folder = 'D:/cnn/'
elif platform.system() == 'Linux': 
    folder = '/home/snu/Downloads/resource'
    

input_x = os.path.join(folder,'X_test_lee10.txt')
input_y = os.path.join(folder,'y_raw.txt')

y_class = 2
TVS = 1200

is_test = True
is_rotate = False
is_clip = False
###############################################################################
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, int(y_class)))
    Yte_onehot = np.zeros((Yte.size, int(y_class)))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot
            
def rotate(image1,image2,angle = None,flip=0):
    img1 = copy.deepcopy(image1)
    img2 = copy.deepcopy(image2)

    if angle is None:
        angle = random.randrange(1,4)
    if flip:
        if type(img1) is list:
            img1.reverse()
        elif type(img1) is np.ndarray:
            img1 = img1.transpose()
        if type(img2) is list:
            img2.reverse()
        elif type(img2) is np.ndarray:
            img2 = img2.transpose()        
    for j in range(angle):
        img1 = [[x[i]for x in img1]  for i in range(len(img1)-1,-1,-1)]
        img2 = [[x[i]for x in img2]  for i in range(len(img2)-1,-1,-1)]

    return np.array(img1),np.array(img2)

def clipimage(image1,image2):
    #x_move = random.randint(0,15)
    #y_move = random.randint(0,15)
    x_move, y_move = 7,7
    return [[image1[y][x] for x in range(x_move,x_move+60)] for y in range(y_move,y_move+60)],[[image2[y][x] for x in range(x_move,x_move+60)] for y in range(y_move,y_move+60)]



Yt, Xtr, Ytr, Xte, Yte, Ytr_onehot, Yte_onehot, Image_idx = [],[],[],[],[],[],[],[]
# load the train and test data
a=open(input_x,'r')
Xt = []
while True:
    temp = a.readline()
    if temp=='':break
    else : exec("Xt.append(%s)" %temp)
a.close()

if not is_test:
    a=open(input_y,'r')
    Yt = []
    exec("Yt=" +a.readline())
    a.close()

    Xt = np.array(Xt)
    Yt = np.array(Yt) 
    
    test_idxs = np.random.permutation(len(Xt))
    Xt = Xt[test_idxs]
    Yt = Yt[test_idxs]

    
    #Train/Valid Sep
    
    Xtr, Ytr, Xte, Yte = Xt[:TVS],Yt[:TVS],Xt[TVS:],Yt[TVS:]
    Image_idx = test_idxs[TVS:]

#Image Augmentation
if is_rotate:
    if not is_test:
        rotated_img =[]
        for i in Xtr:
            rotated_img.append(list(rotate(i[0],i[1],1,0)))
            rotated_img.append(list(rotate(i[0],i[1],2,0)))
            rotated_img.append(list(rotate(i[0],i[1],3,0)))
            rotated_img.append(list(rotate(i[0],i[1],0,1)))
            rotated_img.append(list(rotate(i[0],i[1],1,1)))
            rotated_img.append(list(rotate(i[0],i[1],2,1)))
            rotated_img.append(list(rotate(i[0],i[1],3,1)))
        Xtr = np.append(Xtr,rotated_img,0)
        del rotated_img

        Ytr_temp = copy.deepcopy(Ytr)
        for i in range(7):
            Ytr = np.append(Ytr,Ytr_temp)
        del Ytr_temp
        
if is_clip:
    if not is_test:
        rotated_img =[]
        for i in Xtr:
            rotated_img.append(list(clipimage(i[0],i[1])))
            rotated_img.append(list(clipimage(i[0],i[1])))
            rotated_img.append(list(clipimage(i[0],i[1])))
        Xtr = rotated_img
        
        Ytr_temp = copy.deepcopy(Ytr)
        for i in range(2):
            Ytr = np.append(Ytr,Ytr_temp)
        del Ytr_temp

        Xte_temp=[]
        for i in Xte:
            Xte_temp.append(list(clipimage(i[0],i[1])))
        Xte = Xte_temp
#######################
Xte_temp=[]
for i in Xt:
    Xte_temp.append(list(clipimage(i[0],i[1])))
Xt = Xte_temp
########################        
if not is_test:

    # label data of train and test data, label data is represented by one-hot encoding
    Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)
'''
def load():
    query = train.find()
    Xtr=[]
    Ytr=[]
    for i in query:
        x=[[i['band_1'],i['band_2']]]
        y = [i['is_iceberg']]
        Xtr.extend(x)
        Ytr.extend(y)
    
    query2 = test.find()
    Xte=[]
    for i in query2: 
        xt=[[i['band_1'],i['band_2']]]
        Xte.extend(xt)
        
    return np.array(Xtr), np.array(Ytr), np.array(Xte)
'''

"\ndef load():\n    query = train.find()\n    Xtr=[]\n    Ytr=[]\n    for i in query:\n        x=[[i['band_1'],i['band_2']]]\n        y = [i['is_iceberg']]\n        Xtr.extend(x)\n        Ytr.extend(y)\n    \n    query2 = test.find()\n    Xte=[]\n    for i in query2: \n        xt=[[i['band_1'],i['band_2']]]\n        Xte.extend(xt)\n        \n    return np.array(Xtr), np.array(Ytr), np.array(Xte)\n"

In [3]:
#Xt=[];Yt=[]
with open('testfile_lee10', 'wb') as handle:
    pickle.dump([Xt,Yt,Xtr,Ytr,Xte,Yte,Ytr_onehot, Yte_onehot, Image_idx], handle)
#del Xtr; del Ytr; del Xte